In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('analysis_B20')

enss_all=['b','c','d']

ffs_all=['A20','B20','C20']
js_conn=['j+;conn','j-;conn']

max_mom2_pc,max_mom2_pf=25,0 
n2qpp1s_conn=yum.get_n2qpp1s(max_mom2_pc,max_mom2_pf,noZeroQ=True)

ens2RCs_me=yu.load_pkl('data_aux/RCs.pkl')
ens2msq2pars_jk=yu.load_pkl(f'pkl/analysis_c2pt/reg_ignore/ens2msq2pars_jk.pkl')

# conn; rainbow fit

In [2]:
def encodeTask(task):
    n2qpp1,ff,j=task
    n2q,n2p,n2p1=n2qpp1
    return f"{n2q},{n2p},{n2p1}_{ff}_{j.replace(';',',')}"
def decodeTask(task):
    n2qpp1,ff,j=task.split('_')
    j=j.replace(',',';')
    n2qpp1=tuple([int(ele) for ele in n2qpp1.split(',')])
    return (n2qpp1,ff,j)

# c1s=['all','unequal','equal']; c2s=['err']
c1s=['all']; c2s=['err']
def run(task):
    res={}
    n2qpp1,ff,j=decodeTask(task)
    
    enss=enss_all
    n2q,n2p,n2p1=n2qpp1
    figs=[]
    for ens in enss:
        case2tf2ratio={}
        path=f'/p/project1/ngff/li47/code/projectData/05_moments/doSVD/conn_{ens}_{yu.n2qpp12str(n2qpp1)}.h5'
        tfs_conn=set()
        with h5py.File(path) as f:
            for case in f.keys():
                if case not in case2tf2ratio.keys():
                    case2tf2ratio[case]={}
                for key in f[case].keys():
                    tff,tj,tf=key.split('_'); tf=int(tf); tfs_conn.add(tf)
                    if (tff,tj) != (ff,j):
                        continue
                    case2tf2ratio[case][tf]=f[case][key][:]
        tfs_conn=list(tfs_conn); tfs_conn.sort()
                    
        cases_do=['_'.join([c1,c2]) for c1,c2 in product(c1s,c2s) if len(case2tf2ratio[f'{c1}_{c2}'])!=0]
        eqs=[{'all':'=','unequal':'!=','equal':'='}[case.split('_')[0]] for case in cases_do]
        Z={'j+;conn':'Zqq^s','j-;conn':'Zqq'}[j]
        yunits=[ens2RCs_me[ens][f'{Z}(mu{eq}nu)'] for eq in eqs]
        # yunits=[1 for eq in eqs]
        
        def createDic(ind,case):
            # print(ens,case)
            gett=lambda t:round(t/yu.ens2a[ens])
            def get_tfs(tmin,tmax,dt=1):
                return range(gett(tmin),gett(tmax),dt)
            def cutExtraDiff2tcmins(tmaxExtra,tmaxDiff):
                return yu.cutExtraDiff2tcmins(range(0,gett(tmaxExtra)),range(-gett(tmaxDiff),gett(tmaxDiff)+1),cutBase=2)

            tfmins_1st=tfs_conn
            tcmins_1st=cutExtraDiff2tcmins(0.8,0.4)
            
            tfmins_2st_sum=list(range(8,tfs_conn[-3],2))
            tcmins_2st_sum=cutExtraDiff2tcmins(0.6,0.2)
            
            tfmins_2st=tfmins_2st_sum
            tcmins_2st=tcmins_2st_sum
            
            fittype='2st2step_EFITshare'
            pars_jk_meff2st=[ens2msq2pars_jk[ens][n2p1],ens2msq2pars_jk[ens][n2p]]
            
            tf2ratio=case2tf2ratio[case]
            fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_band')
            fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.9),tcmin=gett(0.2)*2,corrQ=False)
            fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_const')
            fit_const_MA=yu.doMA_3pt(fits_const,tfmin_min=gett(0.9),tcmin_min=gett(0.2)*2)
            fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st_sum,tcmins_2st_sum,unicutQ=2,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_sum')
            fit_sum_MA=yu.doMA_3pt(fits_sum,tcmin_min=gett(0.2)*2)
            fits_2st=yu.doFits_3pt(fittype,tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=pars_jk_meff2st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_2st')
            fit_2st_MA=yu.doMA_3pt(fits_2st,tcmin_min=gett(0.2)*2)
            
            res[(case,'bandfit_WA',ens)]=fit_band_WA[0][:,0]
            res[(case,'const_MA',ens)]=fit_const_MA[0][:,0]
            res[(case,'sum_MA',ens)]=fit_sum_MA[0][:,0]
            res[(case,'2st_MA',ens)]=fit_2st_MA[0][:,0]
                  
            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,fits_2st],
                'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
                'fit_2st_rainbow_midpoint:[fittype,pars_jk_meff2st]':[fittype,pars_jk_meff2st],
                'xyunit':[yu.ens2a[ens],yunits[ind]]
            }
            return dic
        list_dic=[createDic(ind,case) for ind,case in enumerate(cases_do)]

    #     fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint','fit_2st','fit_sum','fit_band','fit_const'],sharey=True)
    #     fig.suptitle(rf'{yu.ens2label[ens]}; n2qpp1={n2qpp1}; $Q^2$={yum.n2qpp12Q2(n2qpp1,ens):.4f} GeV$^2$')
        
    #     for i in range(len(axs)):
    #         axs[i,0].set_ylabel(cases_do[i])
    #     yu.finalizePlot(closeQ=True)
    #     figs.append(fig)

    # yu.makePDF(f'{j}/{yu.n2qpp12str(n2qpp1)}_{ff}',figs,mkdirQ=True)
    
    return res

In [3]:
# n2qpp1s=[(6,6,0),(12,12,0)]
n2qpp1s=n2qpp1s_conn
ffs=ffs_all[1:2]
js=js_conn
enss=enss_all

tasks=[encodeTask((n2qpp1,ff,j)) for n2qpp1 in n2qpp1s for ff in ffs for j in js]
yu.save_txt('data_aux/dat_ignore/analysis_B20_run',tasks)

results=yu.parallelizeTasks(run,tasks)

selection2key2bare_conn=defaultdict(dict)
for task,res in zip(tasks,results):
    n2qpp1,ff,j=decodeTask(task)
    for key,val in res.items():
        case,fittype,ens=key
        selection2key2bare_conn[(case,fittype)][(ens,j,n2qpp1)]=val
        
yu.save_pkl_reg('selection2key2bare_conn',selection2key2bare_conn)

44/44 tasks completed          


# conn; Q2 fit

In [4]:
ens2bare_CK={ens:{} for ens in enss}

divides={'b':1.1167,'c':1.1460,'d':1.1807}
path='data_aux/dat_ignore/connQ2_CK_ignore/isoscalar/'
for file in os.listdir(path):
    t=file.split('_')
    ens,j,ff,msq = t[:4]
    ens=ens.lower(); j={'isos':'j+;conn','isov':'j-;conn'}[j]; msq=int(msq[-2:])

    key=f'{j}_{ff}_{msq}'
    ens2bare_CK[ens][key]=np.load(f'{path}{file}')
path='data_aux/dat_ignore/connQ2_CK_ignore/isovector/'
for file in os.listdir(path):
    t=file.split('_')
    ens,j,ff,msq = t[:4]
    ens=ens.lower(); j={'isos':'j+;conn','isov':'j-;conn'}[j]; msq=int(msq[-2:])

    key=f'{j}_{ff}_{msq}'
    ens2bare_CK[ens][key]=np.load(f'{path}{file}')/divides[ens]

In [5]:
selection2key2bare_conn=yu.load_pkl_reg('selection2key2bare_conn')
selection=('all_err','2st_MA')
key2bare=selection2key2bare_conn[selection]

fig,axs=yu.getFigAxs(2,1,Lrow=4,Lcol=12,sharex=True)
axs[0,0].set_xlim([0,1])
axs[-1,0].set_xlabel(r'$Q^2$ [GeV$^2$]')
xunit=1

enss=enss_all; js=js_conn
for ij,j in enumerate(js):
    ax=axs[ij,0]
    ax.set_ylabel(j)
    for iens,ens in enumerate(enss):
        yunit=ens2RCs_me[ens]['Zqq^s(mu=nu)'] if j in ['j+;conn'] else ens2RCs_me[ens]['Zqq(mu=nu)']
        
        mN=np.mean(ens2msq2pars_jk[ens][0][:,0])
        Q2=np.array([yum.n2qpp12Q2(n2qpp1,ens,mN) for n2qpp1 in n2qpp1s])
        ys=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s])
        mean,err=yu.jackme(ys)
        plt_x=Q2*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors8[iens],fmt=yu.fmts8[iens])
        
        msqs=[n2q for n2q,*_ in n2qpp1s if f'{j}_B20_{n2q}' in ens2bare_CK[ens]]
        Q2=np.array([yum.n2qpp12Q2((msq,msq,0),ens,mN) for msq in msqs])
        ys=np.transpose([ens2bare_CK[ens][f'{j}_B20_{msq}'] for msq in msqs])
        mean,err=yu.jackme(ys)
        plt_x=(Q2+0.005)*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors8[iens],fmt=yu.fmts8[iens],alpha=0.4)

yu.finalizePlot('Q2_conn')